## DCorr-X and MGC-X on fMRI Data

The demo below shows the cross-distance covariance (DCorrX) test and multiscale graph correlation time series (MGC-X) test for independence of time series on fMRI data.

In [ ]:
# Import dependencies.
import numpy as np
import nibabel.cifti2 as ci
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import math

from mgcpy.independence_tests.dcorrx import DCorrX
from mgcpy.independence_tests.mgcx import MGCX

In [ ]:
# Load image - individual 100307.
img = ci.load("rfMRI_REST1_LR_Atlas_hp2000_clean_filt_sm6.HCPMMP.ptseries.nii")
fmri_data = np.array(img.get_fdata())

In [ ]:
# Visualize data, i.e. inspect the first 30 timesteps of each parcel.

# Generate heatmap.
timesteps = 30
displayed_data = np.transpose(fmri_data[range(timesteps),:])
plt.subplots(figsize=(15,10))
ax = sns.heatmap(displayed_data, yticklabels=False)

# Plot parameters.
plt.title('Resting fMRI Signal by Parcel - Individual 100307 LR', fontsize = 20)
plt.ylabel('Parcel', fontsize = 15)
plt.xlabel('Timestep', fontsize = 15)
plt.show()

**We now inspect the dependence between parcels using DCorrX and MGC-X.**

In [ ]:
# Parameters and constants.
p = fmri_data.shape[1] # Number of parcels.
num_bootstrap = 100 # Number of permutations to estimate p-value.
verbose = True # Print output to track progress.
M = 0 # Number of lags in the past to inspect.

In [ ]:
# Function to compute the four matrices with test statistic values and optimal lag/scale.
def compute_dcorrx_matrices(sample_indices):
    dcorrx = DCorrX(max_lag = M)
    
    dcorrx_stat_matrix = np.zeros(p*p).reshape(p, p)
    dcorrx_optimal_lag_matrix = np.zeros(p*p).reshape(p, p)

    n = len(sample_indices)
    for i in range(p):
        # Display progress.
        if verbose:
            if i % math.floor(p / 10):
                print(".", end="")
            else:
                print("\n Estimating dependence structure of parcel", str(i))

        # Off-diagonal entries - measure dependence between parcels.
        for j in range(i):
            if verbose and not i % math.floor(p / 10) and not j % math.floor(p / 36):
                print(".", end='')
            X = fmri_data[sample_indices,i].reshape(n, 1)
            Y = fmri_data[sample_indices,j].reshape(n, 1)

            dcorrx_stat_matrix[i,j], metadata = dcorrx.test_statistic(X, Y)
            dcorrx_optimal_lag_matrix[i,j] = metadata['optimal_lag']
            dcorrx_stat_matrix[j,i], metadata = dcorrx.test_statistic(Y, X)
            dcorrx_optimal_lag_matrix[j,i] = metadata['optimal_lag']

    filename = "dcorrx_stat_matrix_%d.csv" % n
    np.savetxt(filename, dcorrx_stat_matrix, delimiter=",")
    filename = "dcorrx_optimal_lag_matrix_%d.csv" % n
    np.savetxt(filename, dcorrx_optimal_lag_matrix, delimiter=",")

In [ ]:
# Function to compute the four matrices with test statistic values and optimal lag/scale.
def compute_mgcx_matrices(sample_indices):
    mgcx = MGCX(max_lag = M)
    
    mgcx_stat_matrix = np.zeros(p*p).reshape(p, p)
    mgcx_optimal_lag_matrix = np.zeros(p*p).reshape(p, p)
    mgcx_optimal_scale_matrix_X = np.zeros(p*p).reshape(p, p)
    mgcx_optimal_scale_matrix_Y = np.zeros(p*p).reshape(p, p)

    n = len(sample_indices)
    for i in range(p):
        # Display progress.
        if verbose:
            if i % math.floor(p / 10):
                print(".", end="")
            else:
                print("\n Estimating dependence structure of parcel", str(i))

        for j in range(i):
            if verbose and not i % math.floor(p / 10) and not j % math.floor(p / 36):
                print(".", end='')
            X = fmri_data[sample_indices,i].reshape(n, 1)
            Y = fmri_data[sample_indices,j].reshape(n, 1)

            mgcx_stat_matrix[i,j], metadata = mgcx.test_statistic(X, Y)
            mgcx_optimal_lag_matrix[i,j] = metadata['optimal_lag']
            mgcx_optimal_scale_matrix_X[i,j] = metadata['optimal_scale'][0]
            mgcx_optimal_scale_matrix_Y[i,j] = metadata['optimal_scale'][1]
            
            mgcx_stat_matrix[j,i], metadata = mgcx.test_statistic(Y, X)
            mgcx_optimal_lag_matrix[j,i] = metadata['optimal_lag']
            mgcx_optimal_scale_matrix_X[j,i] = metadata['optimal_scale'][0]
            mgcx_optimal_scale_matrix_Y[j,i] = metadata['optimal_scale'][1]

    filename = "mgcx_stat_matrix_%d.csv" % n
    np.savetxt(filename, mgcx_stat_matrix, delimiter=",")
    filename = "mgcx_optimal_lag_matrix_%d.csv" % n
    np.savetxt(filename, mgcx_optimal_lag_matrix, delimiter=",")
    filename = "mgcx_optimal_scale_matrix_X_%d.csv" % n
    np.savetxt(filename, mgcx_optimal_scale_matrix_X, delimiter=",")
    filename = "mgcx_optimal_scale_matrix_Y_%d.csv" % n
    np.savetxt(filename, mgcx_optimal_scale_matrix_Y, delimiter=",")

In [ ]:
# Use 'sample_indices' to control down sampling. If you only want to consider the first 300 timesteps, then run:
# compute_dcorrx_matrices(range(300))
compute_mgcx_matrices(range(300))

In [ ]:
# DCorrX: visualization.

# TO DO: Use labels organizing parcels into regions, and visualize heatmap of p-values and test_statistics.

# Be sure to check the file name from which you want to read!
n = 300
dcorrx_stat_matrix = np.genfromtxt("dcorrx_stat_matrix_%d.csv" % n, delimiter=',')
spread = dcorrx_stat_matrix.max() - dcorrx_stat_matrix.min()
dcorrx_stat_matrix = dcorrx_stat_matrix / spread
plt.subplots(figsize=(18,16))
ax = sns.heatmap(dcorrx_stat_matrix)

# Plot parameters.
plt.title('DCorrX Test Statistic Resting fMRI %d Timesteps - 100307 LR' % n, fontsize = 20)
plt.ylabel('Parcel', fontsize = 17)
plt.xlabel('Parcel', fontsize = 17)
plt.show()

In [ ]:
# MGCX: visualization.

# TO DO: Use labels organizing parcels into regions, and visualize heatmap of p-values and test_statistics.

# Be sure to check the file name from which you want to read!
n = 300
mgcx_stat_matrix = np.genfromtxt("mgcx_stat_matrix_%d.csv" % n, delimiter=',')
spread = mgcx_stat_matrix.max() - mgcx_stat_matrix.min()
mgcx_stat_matrix = mgcx_stat_matrix / spread
plt.subplots(figsize=(18,16))
ax = sns.heatmap(mgcx_stat_matrix)

# Plot parameters.
plt.title('MGCX Test Statistic Resting fMRI %d Timesteps - 100307 LR' % n, fontsize = 20)
plt.ylabel('Parcel', fontsize = 17)
plt.xlabel('Parcel', fontsize = 17)
plt.show()